# ImageNet Classification
>Performance analysis for ImageNet Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [ImageNet,Rooflines,Performance Prediction]
- image: images/imagenet_logo.png

In [16]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 13

csv_path = "./data/cleaned_csv/backup.csv"

# Theoretical Analysis of ImageNet

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input

#first process the following csv's to get clean ready-to-plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(dataframe = pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 
          neural_network = 'imagenet')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for ImageNet classification. The metric used for the theoretical performance is input/second.
Looking at the plot, it becomes clear that prunning along with quantization outputs some of the best performance results.

In [3]:
#hide_input
path_csv = 'data/performance_predictions_imagenet_mnist_cifar_2.csv'
    ## Reading csv file and converting data to (Neural network, Platform, Value)
df = pd.read_csv(path_csv)

df1 = pd.DataFrame()
columns = (df.loc[:, df.columns!='hardw']).columns #select all columns except first
for column in columns:
    df_=pd.melt(df, id_vars=['hardw'], value_vars=column) #melt df1 into a df1 of 2 columns
    df1=pd.concat([df1,df_])
df1.columns= ['y','x','values'] #setting new column names
#replace 0s for NaN values because with 0s the grid doesn't show up
df1['values'] = df1['values'].replace({ 0.0:np.nan})
df_imagenet = dataframe_contains(input_df=df1, column='x', value='GoogleNetv|MobileNetv1|ResNet50|EfficientNet')

df_imagenet.to_csv('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/cleaned_csv/performance_prediction_imagenet.csv', index = False)

#df_cifar10.head(200)
heatmap(df_imagenet, 'pink', 'Performance prediction for ImageNet')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for ImageNet

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [4]:
#hide
print(pd.read_csv('data/overview_experiments_imagenet.csv').to_markdown())

|    | Hardware   | Platform         | ResNet50                  | GoogLeNetV1   | MobileNet   | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:--------------------------|:--------------|:------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | [INT8]*[100%,80%,50%,30%] | INT8          | na          | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | INT8                      | INT8          | na          | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | [INT8]*[100%,80%,50%,30%] | INT8          | INT8        | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | na                        | na            | na          | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | na                        | na            | na          | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | FP16,FP32

In [5]:
#hide_input
%run scripts/script_tables.py 
#get table with the experiments overview
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_imagenet_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [6]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
imagenet_df = maxp_df[maxp_df.NN_Topology.isin(['GNv1','RN50','MNv1','RN50V15']) & maxp_df["top1 [%]"].notna() & maxp_df['lat-comp'].notna()]
bad_precisions = ["FP"+str(i) for i in range(17,24)]
imagenet_df.Precision = imagenet_df.Precision.apply(lambda x: 'FP16' if x in bad_precisions else x)
imagenet_df["hw_quant_prun_net"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor, r.NN_Topology]), axis=1)
imagenet_df["PruningFactor"] = imagenet_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(imagenet_df, "lat-comp", "NN_Topology");
imagenet_df["quant_model"] = imagenet_df.Precision + '_' + imagenet_df.HWType
imagenet_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
imagenet_df["hw_precision_net_prun"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.NN_Topology, str(r.PruningFactor)]), axis=1)

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [17]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp" or row["Op mode"] == "fast" or row["Op mode"] == "slow"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
imagenet_df_v1 = maxp_df[maxp_df.NN_Topology.isin(['GNv1','RN50','MNv1','RN50V15']) & maxp_df['lat-comp'].notna()]
bad_precisions = ["FP"+str(i) for i in range(17,24)]
#this version has the values for ResNet50 v1.5
imagenet_df_v1.Precision = imagenet_df_v1.Precision.apply(lambda x: 'FP16' if x in bad_precisions else x)
imagenet_df_v1["hw_quant_prun_net"] = imagenet_df_v1.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor, r.NN_Topology]), axis=1)
imagenet_df_v1["PruningFactor"] = imagenet_df_v1["PruningFactor"].str.strip("%").astype(float)
norm_by_group(imagenet_df_v1, "lat-comp", "NN_Topology");
imagenet_df_v1["quant_model"] = imagenet_df_v1.Precision + '_' + imagenet_df_v1.HWType
imagenet_df_v1.rename(columns={"top1 [%]": "top1"}, inplace=True)
imagenet_df_v1["hw_precision_net_prun"] = imagenet_df_v1.apply(lambda r: "_".join([r.HWType, r.Precision, r.NN_Topology, str(r.PruningFactor)]), axis=1)

#filling data gaps ommented until know correct values
"""imagenet_df_v1['GOPS'] = imagenet_df_v1.apply(lambda r: 1.14 if r.NN_Topology == 'MNv1' else 
                                          ( 3.13 if r.NN_Topology == 'GNv1' else 
                                           ( 8.2 if r.NN_Topology == 'RN50V15' else 
                                            ( 7.72 if r.NN_Topology == 'RN50' and r.PruningFactor == 100 else 
                                             ( 3.75 if r.NN_Topology == 'RN50' and r.PruningFactor == 50 else 
                                              ( 2 if r.NN_Topology == 'RN50' and r.PruningFactor == 25 else 
                                               ( 1 if r.NN_Topology == 'RN50' and r.PruningFactor == 12.5 else 0 )))))) , axis=1)
"""

imagenet_df_v1.head(200)
#fill in tp-system and tp-cmp
#imagenet_df_v1['tp-system'] = imagenet_df_v1['fps-system'] * imagenet_df_v1['GOPS']
#imagenet_df_v1['tp-comp'] = imagenet_df_v1['fps-comp'] * imagenet_df_v1['GOPS']
#imagenet_df_v1['GOPS'] = imagenet_df_v1['GOPS'] * imagenet_df_v1['batch/thread/stream']
imagenet_df_v1.head(300)

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,NN_Topology,HWType,Precision,Op mode,batch/thread/stream,lat-comp,...,PruningFactor,level,hw_quant_prun_net,norm-lat-comp,quant_model,hw_precision_net_prun
0,RN50V15,EdgeTPU,INT8,fast,1,40.44,...,100.00,l3,EdgeTPU_INT8_100%_RN50V15,0.95,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
1,RN50V15,EdgeTPU,INT8,fast,1,40.59,...,100.00,l3,EdgeTPU_INT8_100%_RN50V15,0.96,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
2,RN50V15,EdgeTPU,INT8,slow,1,42.36,...,100.00,l3,EdgeTPU_INT8_100%_RN50V15,1.00,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
3,RN50V15,EdgeTPU,INT8,slow,1,41.70,...,100.00,l3,EdgeTPU_INT8_100%_RN50V15,0.98,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
273,MNv1,TPU,INT8,slow,1,4.08,...,100.00,l3,TPU_INT8_100%_MNv1,1.00,INT8_TPU,TPU_INT8_MNv1_100.0
274,MNv1,TPU,INT8,fast,1,2.57,...,100.00,l3,TPU_INT8_100%_MNv1,0.63,INT8_TPU,TPU_INT8_MNv1_100.0
403,GNv1,TPU,INT8,slow,1,5.72,...,100.00,l3,TPU_INT8_100%_GNv1,0.01,INT8_TPU,TPU_INT8_GNv1_100.0
404,GNv1,TPU,INT8,fast,1,3.65,...,100.00,l3,TPU_INT8_100%_GNv1,0.00,INT8_TPU,TPU_INT8_GNv1_100.0
437,GNv1,TX2,FP16,maxp,1,6.16,...,100.00,l3,TX2_FP16_100%_GNv1,0.01,FP16_TX2,TX2_FP16_GNv1_100.0
438,GNv1,TX2,FP16,maxp,2,10.62,...,100.00,l3,TX2_FP16_100%_GNv1,0.01,FP16_TX2,TX2_FP16_GNv1_100.0


### Line Plot

In [8]:
#hide_input
dataframe = imagenet_df_v1
sel = alt.selection_multi(fields=["hw_quant_prun_net"], bind="legend")
fig25_dot = alt.Chart(dataframe).mark_point().encode(
    x='lat-comp',
    y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)
fig25_line = alt.Chart(dataframe).mark_line().encode(
    x='lat-comp',
    y='fps-comp',
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)

fig = (fig25_dot+fig25_line).properties(
    title="Latency versus Performance for Pruned and Quantized Imagenet Classifier Variants",
    width=W,
    height=H,
).add_selection(sel).interactive()

fig

alt.LayerChart(...)

### Boxplots

In [9]:
#hide_input 
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df_v1, yaxis="lat-comp", title="Latency by Hardware/Framework and Pruning for imagenet")

alt.FacetChart(...)

In [10]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(imagenet_df_v1, "fps-comp", "Throughput by Hardware/Framework and Pruning for imagenet")

alt.FacetChart(...)

In [11]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df_v1, yaxis="Full_Pwr_W", title="Power Consumption by Hardware/Framework and Pruning for Imagenet")

alt.FacetChart(...)

## Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [12]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph(df= imagenet_df, 
             groupcol= 'hw_precision_net_prun', 
             xcol= 'fps-comp', 
             ycol= 'top1', 
             W= W, 
             H= H, 
             title= "ImageNet Cassification Design Space: Accuracy versus Performance")


alt.LayerChart(...)

In [13]:
#hide
imagenet_df_v1.to_csv('data/cleaned_csv/experimental_data_imagenet.csv', index = False)
imagenet_df_v1.head()

,NN_Topology,HWType,Precision,Op mode,batch/thread/stream,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_quant_prun_net,norm-lat-comp,quant_model,hw_precision_net_prun
0,RN50V15,EdgeTPU,INT8,fast,1,40.44,10.55,24.73,86.53,202.76,nan,nan,nan,nan,1.19,nan,100.00,l3,EdgeTPU_INT8_100%_RN50V15,0.95,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
1,RN50V15,EdgeTPU,INT8,fast,1,40.59,10.59,24.64,86.83,202.05,nan,nan,nan,nan,1.49,nan,100.00,l3,EdgeTPU_INT8_100%_RN50V15,0.96,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
2,RN50V15,EdgeTPU,INT8,slow,1,42.36,10.07,23.61,82.61,193.59,nan,nan,nan,nan,0.96,nan,100.00,l3,EdgeTPU_INT8_100%_RN50V15,1.00,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
3,RN50V15,EdgeTPU,INT8,slow,1,41.70,7.11,23.98,58.31,196.66,nan,nan,nan,nan,1.02,nan,100.00,l3,EdgeTPU_INT8_100%_RN50V15,0.98,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
273,MNv1,TPU,INT8,slow,1,4.08,127.26,244.95,398.31,766.69,69.57,87.71,0.25,0.25,0.46,nan,100.00,l3,TPU_INT8_100%_MNv1,1.00,INT8_TPU,TPU_INT8_MNv1_100.0
